<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">M2.851 · Tipología y ciclo de vida de los datos · PRA1</p>
<p style="margin: 0; text-align:right;">2020-2 · Máster universitario en Ciencia de datos (Data science)</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Estudios de Informática, Multimedia y Telecomunicación</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>


# Práctica 1: Web scraping

 - **https://github.com/sfunesolaria/Pra1WebScraping**

In [ ]:
import scrapy
from scrapy.crawler import CrawlerProcess

In [ ]:
def subcat_url_generator(field, category):
    
    years = [str(yr)[-2:] for yr in range(1993,2022)]
    months = ['01', '02', '03', '04', '05', '06', '07', '08', '09',
              '10', '11', '12']
    
    links = []
    for year in years:
        for month in months:
            link = 'https://arxiv.org/list?archive='
            link += field + '.' + category
            link += '&year=' + year
            link += '&month=' + month
            link += '&submit=Go?skip=0&show=2000'
            links.append(link)

    return links


cs_ai_urls = subcat_url_generator('cs', 'ai')
cs_ai_urls[:50]

Spider should get as `start_urls` an url list, as shown above, to start with.

In [ ]:
import scrapy
from scrapy.crawler import CrawlerProcess


class link_spider(scrapy.Spider):
    name = 'link_spider'

    start_urls = ['https://arxiv.org/list?archive=cs.ai&year=20&month=03&submit=Go?skip=0&show=100']
    found_links = []
    
    base_url = 'https://arxiv.org'

    def parse(self, response):
        for link in response.xpath('//span[@class="list-identifier"]'):
            links = {}
            rel_url = str(link.xpath('.//a/@href').extract_first())
            url = link_spider.base_url + rel_url
            links['url'] = url
            
            self.found_links.append(links)
            
            
if __name__ == "__main__":
    process = CrawlerProcess({
        'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)',
        'LOG_LEVEL': 'ERROR'})
    process.crawl(link_spider)
    spider = next(iter(process.crawlers)).spider
    process.start()

    for event in spider.found_links: print(event)

In [ ]:
import scrapy
from scrapy.crawler import CrawlerProcess

class PythonSpider(scrapy.Spider):
    name = 'pythonspider'

    start_urls = ['https://arxiv.org/abs/2003.00030',]
    found_events = []

    def parse(self, response):
        for event in response.xpath('//div[contains(@class, "leftcolumn")]'):
            event_details = dict()
            
            event_details['subject'] = event.xpath('div[contains(@class, "subheader")]/h1/text()').extract_first()
            
            event_details['title'] = event.xpath('div[contains(@id, "content-inner")]/div/h1[contains(@class, "title mathjax")]/span/following-sibling::text()').extract_first()
            
            event_details['author_1'] = event.xpath('div[contains(@id, "content-inner")]/div/div[contains(@class, "authors")]/a[1]/text()').extract_first()

            event_details['author_2'] = event.xpath('div[contains(@id, "content-inner")]/div/div[contains(@class, "authors")]/a[2]/text()').extract_first()
            
            event_details['author_3'] = event.xpath('div[contains(@id, "content-inner")]/div/div[contains(@class, "authors")]/a[3]/text()').extract_first()
            
            event_details['author_4'] = event.xpath('div[contains(@id, "content-inner")]/div/div[contains(@class, "authors")]/a[4]/text()').extract_first()
            
            event_details['author_5'] = event.xpath('div[contains(@id, "content-inner")]/div/div[contains(@class, "authors")]/a[5]/text()').extract_first()
            
            event_details['abstract'] = event.xpath('div[contains(@id, "content-inner")]/div/blockquote[contains(@class, "abstract mathjax")]/span/following-sibling::text()').extract_first()
            
            self.found_events.append(event_details)

if __name__ == "__main__":
    process = CrawlerProcess({
        'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)',
        'LOG_LEVEL': 'ERROR'})
    process.crawl(PythonSpider)
    spider = next(iter(process.crawlers)).spider
    process.start()

    for event in spider.found_events: print(event)

In [ ]:
import scrapy
from scrapy.crawler import CrawlerProcess

# Double spiders

class data_spider(scrapy.Spider):
    name = 'link_spider'

    found_events = []
    base_url = 'https://arxiv.org'

    def subcat_url_generator(field, category):
        years = [str(yr)[-2:] for yr in range(1993,2022)]
        months = ['01', '02', '03', '04', '05', '06', '07', '08', '09',
              '10', '11', '12']
        links = []
        for year in years:
            for month in months:
                link = 'https://arxiv.org/list?archive='
                link += field + '.' + category
                link += '&year=' + year
                link += '&month=' + month
                link += '&submit=Go?skip=0&show=2000'
                links.append(link)
        return links

    start_urls = subcat_url_generator('cs', 'ai')[:300]

    def parse(self, response):
        for link in response.xpath('//span[@class="list-identifier"]'):
            links = {}
            rel_url = str(link.xpath('.//a/@href').extract_first())
            url = data_spider.base_url + rel_url
            yield scrapy.Request(url, callback=self.parse2)

    def parse2(self, response):
        for event in response.xpath('//div[contains(@class, "leftcolumn")]'):
            event_details = dict()

            event_details['id'] = event.xpath('div[contains(@id, "content-inner")]/div/div[contains(@class, "metatable")]/table/tr/td[contains(@class, "tablecell arxivid")]/span/a/text()').extract_first()

            event_details['category_code'] = event.xpath('div[contains(@id, "content-inner")]/div/div[contains(@class, "metatable")]/table/tr/td[contains(@class, "tablecell arxivid")]/span/a/following-sibling::text()').extract_first().strip()
            
            event_details['category'] = event.xpath('div[contains(@class, "subheader")]/h1/text()').extract_first()
            
            event_details['title'] = event.xpath('div[contains(@id, "content-inner")]/div/h1[contains(@class, "title mathjax")]/span/following-sibling::text()').extract_first()

            event_details['date'] = event.xpath('div[contains(@id, "content-inner")]/div/div/text()').extract_first().replace("\n", "").strip()
            
            event_details['author_1'] = event.xpath('div[contains(@id, "content-inner")]/div/div[contains(@class, "authors")]/a[1]/text()').extract_first()

            event_details['author_2'] = event.xpath('div[contains(@id, "content-inner")]/div/div[contains(@class, "authors")]/a[2]/text()').extract_first()
            
            event_details['author_3'] = event.xpath('div[contains(@id, "content-inner")]/div/div[contains(@class, "authors")]/a[3]/text()').extract_first()
            
            event_details['author_4'] = event.xpath('div[contains(@id, "content-inner")]/div/div[contains(@class, "authors")]/a[4]/text()').extract_first()
            
            event_details['author_5'] = event.xpath('div[contains(@id, "content-inner")]/div/div[contains(@class, "authors")]/a[5]/text()').extract_first()
            
            event_details['summary'] = event.xpath('div[contains(@id, "content-inner")]/div/blockquote[contains(@class, "abstract mathjax")]/span/following-sibling::text()').extract_first().replace("\n", "").strip()
            
            event_details['link'] = event.xpath('div[contains(@id, "content-inner")]/div/div[contains(@class, "metatable")]/table/tr/td[contains(@class, "tablecell arxivid")]/span/a/@href').extract_first()
            
            self.found_events.append(event_details)


if __name__ == "__main__":
    process = CrawlerProcess({
        'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)',
        'LOG_LEVEL': 'ERROR'})
    process.crawl(data_spider)
    process.start()

#    for event in data_spider.found_events: print(event)

In [ ]:
import csv

# create csv

dataset = data_spider.found_events
keys= dataset[0].keys()

try:
    with open("dataset.csv", 'w', newline='') as output_file:
        w = csv.DictWriter(output_file, keys, delimiter=';')
        w.writeheader()
        w.writerows(dataset)
except IOError:
    print("I/O error")

## Covid code

In [ ]:
# Double spiders

class data_spider(scrapy.Spider):
    name = 'data_spider'

    found_events = []

    def search_url_generator(n_results):
        """ Get all results URLs from selected search
    
        n_results (int) -- Number of total results in search
        """
        skip_articles = [str(ini) for ini in range(0, n_results, 25)]
        links = []
        for skip in skip_articles:
            link = 'https://export.arxiv.org/find/all/1/'
            link += 'abs:+AND+vaccines+OR+EXACT+COVID_19+EXACT+SARS_COV-2'
            link += '/0/1/0/all/0/1?'
            link += 'skip=' + skip
            links.append(link)
        return links

    start_urls = search_url_generator(266)
    base_url = 'https://export.arxiv.org'
    
    def parse(self, response):
        # Get all links in search page
        for link in response.xpath('//span[@class="list-identifier"]'):
            links = {}
            rel_url = str(link.xpath('.//a/@href').extract_first())
            url = data_spider.base_url + rel_url
            yield scrapy.Request(url, callback=self.parse2)


    def parse2(self, response):
        # Get elements of interest in article page and store
        for event in response.xpath('//div[contains(@class, "leftcolumn")]'):
            event_details = dict()

            event_details['id'] = event.xpath('div[contains(@id, "content-inner")]/div/div[contains(@class, "metatable")]/table/tr/td[contains(@class, "tablecell arxivid")]/span/a/text()').extract_first()

            event_details['category_code'] = event.xpath('div[contains(@id, "content-inner")]/div/div[contains(@class, "metatable")]/table/tr/td[contains(@class, "tablecell arxivid")]/span/a/following-sibling::text()').extract_first().strip()
            
            event_details['category'] = event.xpath('div[contains(@class, "subheader")]/h1/text()').extract_first()
            
            event_details['title'] = event.xpath('div[contains(@id, "content-inner")]/div/h1[contains(@class, "title mathjax")]/span/following-sibling::text()').extract_first()

            event_details['date'] = event.xpath('div[contains(@id, "content-inner")]/div/div/text()').extract_first().replace("\n", "").strip()
            
            event_details['author_1'] = event.xpath('div[contains(@id, "content-inner")]/div/div[contains(@class, "authors")]/a[1]/text()').extract_first()

            event_details['author_2'] = event.xpath('div[contains(@id, "content-inner")]/div/div[contains(@class, "authors")]/a[2]/text()').extract_first()
            
            event_details['author_3'] = event.xpath('div[contains(@id, "content-inner")]/div/div[contains(@class, "authors")]/a[3]/text()').extract_first()
            
            event_details['author_4'] = event.xpath('div[contains(@id, "content-inner")]/div/div[contains(@class, "authors")]/a[4]/text()').extract_first()
            
            event_details['author_5'] = event.xpath('div[contains(@id, "content-inner")]/div/div[contains(@class, "authors")]/a[5]/text()').extract_first()
            
            event_details['summary'] = event.xpath('div[contains(@id, "content-inner")]/div/blockquote[contains(@class, "abstract mathjax")]/span/following-sibling::text()').extract_first().replace("\n", "").strip()
            
            event_details['link'] = event.xpath('div[contains(@id, "content-inner")]/div/div[contains(@class, "metatable")]/table/tr/td[contains(@class, "tablecell arxivid")]/span/a/@href').extract_first()
            
            self.found_events.append(event_details)


if __name__ == "__main__":
    process = CrawlerProcess({
        'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)',
        'LOG_LEVEL': 'ERROR'})
    process.crawl(data_spider)
    process.start()

In [ ]:
# Save dataset into csv

dataset = data_spider.found_events
keys = dataset[0].keys()

try:
    with open("dataset_covid.csv", 'w', newline='') as output_file:
        w = csv.DictWriter(output_file, keys, delimiter=';')
        w.writeheader()
        w.writerows(dataset)
except IOError:
    print("I/O error")